# Run Savercat with highly vairiable genes

In [1]:
#Import Packages
import random
import os
import numpy as np
import scanpy as sc
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.utils.vis_utils import plot_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, BatchNormalization, LeakyReLU, Lambda
from tensorflow.keras import Model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, scale
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as pl

In [2]:
base_name = os.path.basename(os.getcwd())
print(base_name)
print(sc.__version__)
sc.settings.verbosity = 3  
sc.logging.print_versions()

bin
1.5.1
scanpy==1.5.1 anndata==0.7.4 umap==0.4.6 numpy==1.20.1 scipy==1.4.1 pandas==1.0.3 scikit-learn==0.23.1 statsmodels==0.11.1 python-igraph==0.8.2 leidenalg==0.8.1


# 1. Load data

In [3]:
adata = sc.read_h5ad('../data/adata_subsample_hvg.h5ad')
print(adata)

AnnData object with n_obs × n_vars = 29259 × 2668
    obs: 'Cycle', 'patient'


# 2. Savercat preprocess

In [4]:
# import utils functions in utils_0509.py
from utils_0509 import *

In [5]:
predict_key = 'Cycle' # the name of the cell-level label to be predicted
batch_key = 'patient' # the name of the cell-level label to be adjusted for

In [6]:
# savercat preprocessing step
adata = savercat_preprocess(adata, predict_key=predict_key, adjust_key=batch_key)
adata

normalizing by total count per cell
    finished (0:00:01): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)


AnnData object with n_obs × n_vars = 29259 × 2668
    obs: 'Cycle', 'patient', 'n_counts', 'size_factors'
    var: 'mean', 'std'
    uns: 'log1p'
    obsm: 'saver_targetL', 'B_raw', 'B', 'loglib', 'saver_batch'

# 3. Build the model

In [7]:
# import network buiding functions in network_0509.py
from network_0509 import * 

In [8]:
# if train on highly variable genes, then keep enc=(256, 256, 128), dec=(128, 256, 256)
# leave all the parameters unchanged
SAVER_net = CVAE(x_input_size = adata.n_vars, # number of genes
                 b_input_size = adata.obsm['saver_batch'].shape[1], # number of batches including lib-size
                 lb_input_size = adata.obsm['saver_targetL'].shape[1], # number of labels to predict
                 enc = (256, 256, 128), # dim of the encoder
                 dec = (128, 256, 256), # dim of the decoder
                 latent_k = 30) # dimension of the low-dimensional latent space
SAVER_net.build()
SAVER_net.compile_model(pred_weight=1, kl_weight=1)

# 4. Initialize the model

In [9]:
# no need to modify this block
# label guided initialization step
loss = SAVER_net.model_initialize(adata, fit_verbose=1)

Epoch 1/300
412/412 [==============================] - 12s 30ms/step - loss: 812.0385 - pred_loss: 44.5037 - kl_loss: 13.4647 - recon_loss: 754.0351 - val_loss: 726.8876 - val_pred_loss: 31.4546 - val_kl_loss: 17.4698 - val_recon_loss: 677.8963
Epoch 2/300
412/412 [==============================] - 12s 30ms/step - loss: 711.2430 - pred_loss: 23.0290 - kl_loss: 11.3375 - recon_loss: 676.8601 - val_loss: 707.9414 - val_pred_loss: 25.3956 - val_kl_loss: 12.4498 - val_recon_loss: 670.0816
Epoch 3/300
412/412 [==============================] - 12s 30ms/step - loss: 695.8997 - pred_loss: 15.3168 - kl_loss: 10.5742 - recon_loss: 669.9968 - val_loss: 701.5477 - val_pred_loss: 24.5124 - val_kl_loss: 11.0520 - val_recon_loss: 665.9819
Epoch 4/300
412/412 [==============================] - ETA: 0s - loss: 687.2782 - pred_loss: 10.9851 - kl_loss: 10.3054 - recon_loss: 665.9832
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
412/412 [==============================] 

In [10]:
# fill in the directory where you want to save the file
# 'weights_step1.h5' is the file name
SAVER_net.model.save_weights('../data/weights_init.h5') 

# 5. Fine-tune the model

In [11]:
# if train on highly variable genes, then keep enc=(256, 256, 128), dec=(128, 256, 256)
# leave all the parameters unchanged
# same as block 8 but use the weight you just saved
SAVER_net = CVAE(x_input_size = adata.n_vars,
                 b_input_size = adata.obsm['saver_batch'].shape[1],
                 lb_input_size = adata.obsm['saver_targetL'].shape[1],
                 enc = (256, 256, 128),
                 dec = (128, 256, 256),
                 latent_k = 30)
SAVER_net.build()
SAVER_net.load_weights('../data/weights_init.h5') # fill in the weight file you just saved
SAVER_net.compile_model(pred_weight=0., kl_weight=1)

In [12]:
# no need to modify this block
# train savercat model which do the dimension reduction
loss = SAVER_net.model_finetune(adata, fit_verbose=1)

Epoch 1/300
412/412 [==============================] - 13s 31ms/step - loss: 672.8130 - pred_loss: 15.3398 - kl_loss: 9.8709 - recon_loss: 662.9423 - val_loss: 670.8999 - val_pred_loss: 42.0698 - val_kl_loss: 10.6762 - val_recon_loss: 660.2236
Epoch 2/300
412/412 [==============================] - 12s 30ms/step - loss: 668.9038 - pred_loss: 32.3026 - kl_loss: 9.7614 - recon_loss: 659.1422 - val_loss: 667.9171 - val_pred_loss: 48.4594 - val_kl_loss: 9.8714 - val_recon_loss: 658.0456
Epoch 3/300
412/412 [==============================] - 14s 33ms/step - loss: 666.2770 - pred_loss: 41.5775 - kl_loss: 9.7577 - recon_loss: 656.5198 - val_loss: 665.9383 - val_pred_loss: 51.4322 - val_kl_loss: 10.0053 - val_recon_loss: 655.9332
Epoch 4/300
412/412 [==============================] - 14s 34ms/step - loss: 664.5171 - pred_loss: 46.4905 - kl_loss: 9.7208 - recon_loss: 654.7960 - val_loss: 664.3216 - val_pred_loss: 58.5913 - val_kl_loss: 9.6128 - val_recon_loss: 654.7088
Epoch 5/300
412/412 [=====

412/412 [==============================] - 14s 35ms/step - loss: 646.9056 - pred_loss: 139.8928 - kl_loss: 10.9849 - recon_loss: 635.9213 - val_loss: 655.2656 - val_pred_loss: 136.4519 - val_kl_loss: 10.7354 - val_recon_loss: 644.5303
Epoch 35/300
412/412 [==============================] - 14s 33ms/step - loss: 646.4771 - pred_loss: 139.5728 - kl_loss: 11.0205 - recon_loss: 635.4564 - val_loss: 655.6636 - val_pred_loss: 137.5453 - val_kl_loss: 10.8075 - val_recon_loss: 644.8561
Epoch 36/300
412/412 [==============================] - 13s 32ms/step - loss: 646.4547 - pred_loss: 139.9367 - kl_loss: 11.0462 - recon_loss: 635.4081 - val_loss: 655.6312 - val_pred_loss: 133.9467 - val_kl_loss: 10.7918 - val_recon_loss: 644.8394
Epoch 37/300
411/412 [============================>.] - ETA: 0s - loss: 646.2880 - pred_loss: 140.7673 - kl_loss: 11.1212 - recon_loss: 635.1669
Epoch 00037: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
412/412 [==============================] - 

In [13]:
# predict the low-dimensional embedding for all the cells, and save to a csv file
meta_df_train = adata.obs
z_train = SAVER_net.extra_models['mean_out'].predict([adata.X, adata.obsm['saver_batch']])
z_df = pd.DataFrame(z_train, 
                    index = meta_df_train.index,
                    columns = ['saver{}'.format(i+1) for i in range(SAVER_net.latent_k)])
z_df.to_csv('../data/lowdim_savercat_hvg.csv') # where you want to save the low-dimensional embeddings learned by SAVERCAT